## Entrenamiento Ridge

In [ ]:
import sys, pathlib
import pandas as pd
import joblib
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

# Añadir src/ al path para importar cfg
PROJECT_ROOT = pathlib.Path().resolve().parents[1]
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src import config as cfg

In [ ]:
# Cargar dataset procesado de XGB
df = joblib.load(cfg.DATA / 'processed' / 'xgb_data.pkl')
print(f'✅ Datos cargados: {len(df):,} muestras')

# Separar train/test por fecha
split_date = df['date'].quantile(0.8)
df_train = df[df['date'] <= split_date].copy()
df_test  = df[df['date'] > split_date].copy()
print(f'📊 Train: {len(df_train):,} | Test: {len(df_test):,}')

In [ ]:
features = ['ret_1d','ret_5d','vol_5d','momentum']
models = {}
mae_scores = {}
for ticker in df['ticker'].unique():
    tr = df_train[df_train['ticker'] == ticker]
    te = df_test[df_test['ticker'] == ticker]
    X_tr, y_tr = tr[features], tr['target_5d']
    X_te, y_te = te[features], te['target_5d']
    model = Ridge(alpha=1.0)
    model.fit(X_tr, y_tr)
    pred = model.predict(X_te)
    mae = mean_absolute_error(y_te, pred)
    models[ticker] = model
    mae_scores[ticker] = mae
    print(f'✅ {ticker:5} | MAE: {mae:.5f} | Train: {len(X_tr):4} | Test: {len(X_te):4}')

In [ ]:
joblib.dump(models, cfg.MODELS / 'ridge.pkl')
print('💾 Modelos guardados en models/ridge.pkl')